# This is my first plotting script for CxAODReaderbbtt output

In [1]:
#!/usr/etc/env python
import ROOT

Welcome to JupyROOT 6.22/02


In [2]:
# Select variable
def SelectVariable(list_of_hist, variable):
    """
    input: list of histogram, variable
    output: list contains all histo with variable
    """
    list_return = []
    for _hist in list_of_hist:

        _a = _hist.split('_')
        # print(_a)
        if _a[-1:][0] == variable:
            list_return.append(_hist)

    return list_return


def splitRegion(list_of_name):
    """
    input :such as [a,b,c,d,e,f]
    output: b_c_d_e
    """

    return list_of_name[1]+'_'+list_of_name[2]+'_'+list_of_name[3]+'_'+list_of_name[4]

# Get All histograms name in Preselection


def GetAllKeysInPreselection(tPre):
    """
     input: TPreselection
     return: All TH1F in TPreselection as a list

    """
    olist = []
    for key in tPre.GetListOfKeys():
        if key.GetName() == 'Systematics':
            continue
        olist.append(key.GetName())
    return olist

In [3]:
# path for Hist-OUTPUT
dirHistOUTPUT = '/home/xuzifeng/HistxAOD/submitDir/hist-OUTPUT.root'
dir_save_figure='/home/xuzifeng/HistxAOD/submitDir/figure/'

# Select histograms we need!
region_tag = '2tag2pjet_0ptv_LL_OS'

# Get Preselection
file = ROOT.TFile(dirHistOUTPUT)
tPreselection = file.Get('Preselection')

In [4]:
# Get all keys of histograms and save them in a list!

# Get All Keys!
ALLHIST=GetAllKeysInPreselection(tPreselection)
print(ALLHIST)

['data_2tag2pjet_0ptv_LL_SS_mMMC', 'data_2tag2pjet_0ptv_LL_SS_dRBB', 'data_2tag2pjet_0ptv_LL_SS_dRTauTau', 'data_2tag2pjet_0ptv_LL_SS_Tau1Eta', 'data_2tag2pjet_0ptv_LL_SS_Tau0Eta', 'data_2tag2pjet_0ptv_LL_SS_Tau1Pt', 'data_2tag2pjet_0ptv_LL_SS_pTB1', 'data_2tag2pjet_0ptv_LL_SS_pTB0', 'data_2tag2pjet_0ptv_LL_SS_Jet0Pt', 'data_2tag2pjet_0ptv_LL_SS_Yield', 'data_1tag2pjet_0ptv_LL_OS_pTB1', 'Fake_2tag2pjet_0ptv_LL_OS_Tau1Eta', 'Fake_1tag2pjet_0ptv_LL_SS_mHH', 'Fake_1tag2pjet_0ptv_LL_SS_pTB0', 'Fake_1tag2pjet_0ptv_LL_SS_Tau0Eta', 'Fake_1tag2pjet_0ptv_LL_SS_mMMC', 'Fake_1tag2pjet_0ptv_LL_SS_mJJVBF', 'Fake_1tag2pjet_0ptv_LL_SS_Tau1Eta', 'Fake_1tag2pjet_0ptv_LL_SS_Tau1Pt', 'Fake_1tag2pjet_0ptv_LL_SS_dRTauTau', 'data_1tag2pjet_0ptv_LL_OS_Tau1Pt', 'Fake_1tag2pjet_0ptv_LL_SS_Tau0Pt', 'data_1tag2pjet_0ptv_LL_OS_dEtaVBF', 'data_1tag2pjet_0ptv_LL_SS_dEtaVBF', 'data_2tag2pjet_0ptv_LL_SS_nJets', 'Fake_1tag2pjet_0ptv_LL_SS_Yield', 'data_1tag2pjet_0ptv_LL_SS_dRBB', 'Fake_2tag2pjet_0ptv_LL_SS_mJJVBF', 'F

In [5]:
# Get All Keys and sve them in table
dic_hist = {'Process': [], 'Region': [], 'Variable': []}

In [6]:
# Just GetKeys and save them in a dictionary

for h_name in ALLHIST:
    h_name = h_name.split('_')
    process_name = h_name[0]
    region_name = splitRegion(h_name)
    variable_name = h_name[-1]

    if process_name not in dic_hist['Process']:
        dic_hist['Process'].append(process_name)
    if region_name not in dic_hist['Region']:
        dic_hist['Region'].append(region_name)
    if variable_name not in dic_hist['Variable']:
        dic_hist['Variable'].append(variable_name)
print(dic_hist)

{'Process': ['data', 'Fake', 'ggFHtautau', 'VBFHtautau', 'WHtautau', 'hhttbb', 'hhttbbFTapprox', 'stops', 'stopt', 'stopWt', 'ttbar', 'ttbarSFTF', 'ttbarSFFF', 'ttbarSFFT', 'ttbarAF2', 'ttbarAF2FT', 'ttbarAF2TF', 'ttbarAF2FF', 'ttbarAMCP8', 'ttbarAMCP8FT', 'ttbarAMCP8TF', 'ttbarAMCP8FF', 'ttbarPP8RadHiFF', 'ttbarPP8RadHiFT', 'ttbarPP8RadHiTF', 'ttbarPP8RadHi', 'ttbarPH7', 'ttbarPH7TF', 'ttbarPH7FT', 'ttbarPH7FF', 'WHbb', 'ZHbb', 'W', 'Wtt', 'WW', 'WZ', 'Hhhbbtautau1000', 'Hhhbbtautau1100', 'Hhhbbtautau1200', 'Hhhbbtautau1400', 'Hhhbbtautau1600', 'Hhhbbtautau1800', 'Hhhbbtautau2000', 'Hhhbbtautau2500', 'Hhhbbtautau251', 'Hhhbbtautau260', 'Hhhbbtautau280', 'Hhhbbtautau3000', 'Hhhbbtautau300', 'Hhhbbtautau325', 'Hhhbbtautau350', 'Hhhbbtautau400', 'Hhhbbtautau450', 'Hhhbbtautau500', 'Hhhbbtautau550', 'Hhhbbtautau600', 'Hhhbbtautau700', 'Hhhbbtautau800', 'Hhhbbtautau900', 'Zbl', 'Zbb', 'Zbc', 'Zcl', 'Zl', 'Zttcc', 'Zttl', 'Zttbc', 'Zttbb', 'Zttbl', 'Zttcl', 'ZZ'], 'Region': ['2tag2pjet_0ptv

In [7]:
# Now stack all histogram needed and draw

# Remember check if the histogram we generate in the ALLHIST
# Generate our histogramlist
data_list = []
fake_list = []
process_list = []

In [8]:
# Fill process_list
for _process in dic_hist['Process']:
    if _process == 'data':
        continue
    if _process == 'Fake':
        continue
    for _region in dic_hist['Region']:
        if _region == region_tag:
            for _variable in dic_hist['Variable']:
                a = _process + '_'+_region+'_'+_variable
                if a in ALLHIST:
                    process_list.append(a)
print(process_list)

['ggFHtautau_2tag2pjet_0ptv_LL_OS_mMMC', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_dRBB', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_dRTauTau', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Tau1Eta', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Tau0Eta', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Tau1Pt', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_pTB1', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_pTB0', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Jet0Pt', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Yield', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_mHH', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_mJJVBF', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Tau0Pt', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_dEtaVBF', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_nJets', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_mBB', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_Jet1Pt', 'ggFHtautau_2tag2pjet_0ptv_LL_OS_MET', 'VBFHtautau_2tag2pjet_0ptv_LL_OS_mMMC', 'VBFHtautau_2tag2pjet_0ptv_LL_OS_dRBB', 'VBFHtautau_2tag2pjet_0ptv_LL_OS_dRTauTau', 'VBFHtautau_2tag2pjet_0ptv_LL_OS_Tau1Eta', 'VBFHtautau_2tag2pjet_0ptv_LL_OS_Tau0Eta', 'VBFHtautau_2tag2pjet_0ptv_LL_OS_Tau1Pt', 'VBFH

In [9]:
# Fill fake_list
for _process in dic_hist['Process']:
    if not(_process == 'Fake'):
        continue
    for _region in dic_hist['Region']:
        if _region == region_tag:
            for _variable in dic_hist['Variable']:
                a = _process + '_'+_region+'_'+_variable
                if a in ALLHIST:
                    fake_list.append(a)
print(fake_list)

['Fake_2tag2pjet_0ptv_LL_OS_mMMC', 'Fake_2tag2pjet_0ptv_LL_OS_dRBB', 'Fake_2tag2pjet_0ptv_LL_OS_dRTauTau', 'Fake_2tag2pjet_0ptv_LL_OS_Tau1Eta', 'Fake_2tag2pjet_0ptv_LL_OS_Tau0Eta', 'Fake_2tag2pjet_0ptv_LL_OS_Tau1Pt', 'Fake_2tag2pjet_0ptv_LL_OS_pTB1', 'Fake_2tag2pjet_0ptv_LL_OS_pTB0', 'Fake_2tag2pjet_0ptv_LL_OS_Jet0Pt', 'Fake_2tag2pjet_0ptv_LL_OS_Yield', 'Fake_2tag2pjet_0ptv_LL_OS_mHH', 'Fake_2tag2pjet_0ptv_LL_OS_mJJVBF', 'Fake_2tag2pjet_0ptv_LL_OS_Tau0Pt', 'Fake_2tag2pjet_0ptv_LL_OS_dEtaVBF', 'Fake_2tag2pjet_0ptv_LL_OS_nJets', 'Fake_2tag2pjet_0ptv_LL_OS_mBB', 'Fake_2tag2pjet_0ptv_LL_OS_Jet1Pt', 'Fake_2tag2pjet_0ptv_LL_OS_MET']


In [10]:
# Fill data_list
for _process in dic_hist['Process']:
    if not(_process == 'data'):
        continue
    for _region in dic_hist['Region']:
        if _region == region_tag:
            for _variable in dic_hist['Variable']:
                a = _process + '_'+_region+'_'+_variable
                if a in ALLHIST:
                    data_list.append(a)
print(data_list)

['data_2tag2pjet_0ptv_LL_OS_mMMC', 'data_2tag2pjet_0ptv_LL_OS_dRBB', 'data_2tag2pjet_0ptv_LL_OS_dRTauTau', 'data_2tag2pjet_0ptv_LL_OS_Tau1Eta', 'data_2tag2pjet_0ptv_LL_OS_Tau0Eta', 'data_2tag2pjet_0ptv_LL_OS_Tau1Pt', 'data_2tag2pjet_0ptv_LL_OS_pTB1', 'data_2tag2pjet_0ptv_LL_OS_pTB0', 'data_2tag2pjet_0ptv_LL_OS_Jet0Pt', 'data_2tag2pjet_0ptv_LL_OS_Yield', 'data_2tag2pjet_0ptv_LL_OS_mHH', 'data_2tag2pjet_0ptv_LL_OS_mJJVBF', 'data_2tag2pjet_0ptv_LL_OS_Tau0Pt', 'data_2tag2pjet_0ptv_LL_OS_dEtaVBF', 'data_2tag2pjet_0ptv_LL_OS_nJets', 'data_2tag2pjet_0ptv_LL_OS_mBB', 'data_2tag2pjet_0ptv_LL_OS_Jet1Pt', 'data_2tag2pjet_0ptv_LL_OS_MET']


In [11]:
dic_THStack={}
dic_TCanvas={}
for _variable in dic_hist['Variable']:
    
    # Create Canvas and THStacks!
    dic_TCanvas[_variable]=ROOT.TCanvas()
    dic_THStack[_variable]=ROOT.THStack(_variable, _variable)
print(dic_THStack)
print(dic_TCanvas)

{'mMMC': <cppyy.gbl.THStack object at 0x556b7d462540>, 'dRBB': <cppyy.gbl.THStack object at 0x556b7d48f230>, 'dRTauTau': <cppyy.gbl.THStack object at 0x556b7d480970>, 'Tau1Eta': <cppyy.gbl.THStack object at 0x556b7d4673f0>, 'Tau0Eta': <cppyy.gbl.THStack object at 0x556b7c0ceb00>, 'Tau1Pt': <cppyy.gbl.THStack object at 0x556b7d462740>, 'pTB1': <cppyy.gbl.THStack object at 0x556b7d462690>, 'pTB0': <cppyy.gbl.THStack object at 0x556b7d481160>, 'Jet0Pt': <cppyy.gbl.THStack object at 0x556b7d460f80>, 'Yield': <cppyy.gbl.THStack object at 0x556b7d463f00>, 'mHH': <cppyy.gbl.THStack object at 0x556b7d4422b0>, 'mJJVBF': <cppyy.gbl.THStack object at 0x556b7d4805c0>, 'Tau0Pt': <cppyy.gbl.THStack object at 0x556b7d49bd60>, 'dEtaVBF': <cppyy.gbl.THStack object at 0x556b7d4950b0>, 'nJets': <cppyy.gbl.THStack object at 0x556b7d461520>, 'mBB': <cppyy.gbl.THStack object at 0x556b7d47a2f0>, 'Jet1Pt': <cppyy.gbl.THStack object at 0x556b7d4689d0>, 'MET': <cppyy.gbl.THStack object at 0x556b7d4631d0>}
{'mMM

In [12]:

# print(process_list)
# print(SelectVariable(process_list, 'mBB'))
for _variable in dic_hist['Variable']:

    # Collecting histogram with differenct variable
    hist_data = SelectVariable(data_list, _variable)
    hist_process = SelectVariable(process_list, _variable)
    hist_fake = SelectVariable(fake_list, _variable)
    hs_v = ROOT.THStack(_variable, 'test stacked histograms')
    for _hist in hist_process:
        _histogram = tPreselection.Get(_hist)
        dic_THStack[_variable].Add(_histogram)
    for _hist in hist_fake:
        _histogram = tPreselection.Get(_hist)
        dic_THStack[_variable].Add(_histogram)
    dic_TCanvas[_variable].cd()
    dic_THStack[_variable].Draw('HIST')
    dic_TCanvas[_variable].Update()
    dic_TCanvas[_variable].SaveAs(_variable+'.pdf')

Info in <TCanvas::Print>: pdf file mMMC.pdf has been created
Info in <TCanvas::Print>: pdf file dRBB.pdf has been created
Info in <TCanvas::Print>: pdf file dRTauTau.pdf has been created
Info in <TCanvas::Print>: pdf file Tau1Eta.pdf has been created
Info in <TCanvas::Print>: pdf file Tau0Eta.pdf has been created
Info in <TCanvas::Print>: pdf file Tau1Pt.pdf has been created
Info in <TCanvas::Print>: pdf file pTB1.pdf has been created
Info in <TCanvas::Print>: pdf file pTB0.pdf has been created
Info in <TCanvas::Print>: pdf file Jet0Pt.pdf has been created
Info in <TCanvas::Print>: pdf file Yield.pdf has been created
Info in <TCanvas::Print>: pdf file mHH.pdf has been created
Info in <TCanvas::Print>: pdf file mJJVBF.pdf has been created
Info in <TCanvas::Print>: pdf file Tau0Pt.pdf has been created
Info in <TCanvas::Print>: pdf file dEtaVBF.pdf has been created
Info in <TCanvas::Print>: pdf file nJets.pdf has been created
Info in <TCanvas::Print>: pdf file mBB.pdf has been created
Inf